In [1]:
import os
import torch

from pytorch_lightning import Trainer, seed_everything


In [ ]:

from bci_aic3.paths import (
    RAW_DATA_DIR,
    MI_CONFIG_PATH,
    SSVEP_CONFIG_PATH,
    MI_RUNS_DIR,
)
from bci_aic3.models.eegnet import EEGNet
from bci_aic3.util import rec_cpu_count

In [3]:
from bci_aic3.train import train_model

In [4]:
# Code necessary to create reproducible runs
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


Seed set to 42


In [ ]:


trainer, model = train_model(model=EEGNet,
                             config_path=MI_CONFIG_PATH)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
p:\Programming\AIC3\repo\bci_aic3\.venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory P:\Programming\AIC3\repo\bci_aic3\checkpoints\MI exists and is not empty.

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | EEGNet             | 36.2 K | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | val_accuracy   | MulticlassAccuracy | 0      | train
4 | train_f1       | MulticlassF1Score  | 0      | train
5 | val_f1         | MulticlassF1Score  | 0      | train
--------------------------------------------------------------
36.2 K    Trainable params
0         Non-trainable params
36.2 K    Total params
0.145     Total estimated model params size (MB)
20        Modules in train 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

p:\Programming\AIC3\repo\bci_aic3\.venv\Lib\site-packages\torch\nn\modules\conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Convolution.cpp:1037.)
  return F.conv2d(


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
from torch.nn.utils import parametrize

# Remove all parametrizations from the model
def remove_parametrizations(model):
    for name, module in model.named_modules():
        if hasattr(module, 'parametrizations'):
            # Remove all parametrizations for this module
            for param_name in list(module.parametrizations.keys()):
                parametrize.remove_parametrizations(module, param_name)
    return model

In [ ]:
unparamed_model = remove_parametrizations(model)

In [ ]:
scripted_model = torch.jit.script(unparamed_model)
torch.save(scripted_model, MI_RUNS_DIR / "model_scripted.pt") # Export to TorchScript

In [ ]:

torch.save(model_trace, MI_RUNS_DIR / 'model_trace.pt') # Save

In [ ]:
loaded_model = torch.load(MI_RUNS_DIR / "mi_scripted.pt", weights_only=False)

In [ ]:
loaded_model = torch.jit.load(MI_RUNS_DIR / "model_scripted.pt")

In [ ]:
from bci_aic3.inference import create_inference_data_loader, load_and_preprocess_for_inference
from bci_aic3.inference import predict_batch


In [ ]:
test_dataset = load_and_preprocess_for_inference("test.csv", 
                                                 base_path=RAW_DATA_DIR,
                                                 task_type="MI")

test_loader = create_inference_data_loader(test_dataset)

In [ ]:
unparamed_model = remove_parametrizations(loaded_model)

In [ ]:
loaded_model.eval()
predictions = []
device = "cpu"

with torch.no_grad():
    for x_batch in test_loader:
        x_batch = x_batch[0]
        x_batch = x_batch.to(device)
        outputs = loaded_model(x_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(preds.tolist())

In [ ]:
predictions